## Dataset Consolidation Script

Objective of this script is to consolidate Tom's Glassdoor data, Pengjin's Adzuna data, and the university-provided Adzuna dataset into one finalised, clean ore Dataset: ***master_cleaned_job_listings.csv***

* This required a systematic approach to handle inconsistencies in data structure, harmonize temporal data, and address significant volumes of missing salary information.

In [1]:
#Step 1: Load all cleaned datasets

import pandas as pd
import numpy as np

# Load your data
df_pengjin_adzuna = pd.read_csv('pengjin_adzuna_cleaned_job_listings_final.csv')
df_uni_adzuna = pd.read_csv('uni_adzuna_cleaned_job_listings_final.csv')
df_tom_glassdoor = pd.read_csv('tom_glassdoor_cleaned_job_listings_final.csv')

In [2]:
#Step 2: Create a 'Source' and 'Author' Column

df_pengjin_adzuna['source'] = 'Adzuna'
df_uni_adzuna['source'] = 'Adzuna'
df_tom_glassdoor['source'] = 'Glassdoor'

df_pengjin_adzuna['author'] = 'Pengjin'
df_uni_adzuna['author'] = 'UoB'
df_tom_glassdoor['author'] = 'Tom'

print(df_pengjin_adzuna.shape)
print(df_uni_adzuna.shape)
print(df_tom_glassdoor.shape)

(916, 21)
(8530, 21)
(651, 21)


***
### Step 3: Handling Missing "date posted" for Glassdoor Data

* Glassdoor job listings do not contain "date posted" information like Adzuna does.

* We do know however the date that the data was scraped as well as that Glassdoor typically make companies close job listings within 30 or 60 days from the posting date *(will use median of 45 days)*

* We will use random imputation below to generate missing "date posted" data in order to use the Glassdoor data for Time Series analysis

* An outline of why decided to use random imputation is below.


In [3]:
# Step 3: Generate "date posted" for Glassdoor data using Random Imputation

# Define the start and end dates of window
start_date = pd.to_datetime('2025-04-30') # 45 days before end date
end_date = pd.to_datetime('2025-06-18') # Last date data was scraped

# Calculate the number of days in the period
n_days = (end_date - start_date).days

# Generate random date offsets for each row in the Glassdoor dataframe
# A random seed makes the "random" numbers predictable and your results reproducible
np.random.seed(42) 
random_days = np.random.randint(0, n_days, size=len(df_tom_glassdoor))
random_dates = start_date + pd.to_timedelta(random_days, unit='d')

# Assign the new dates to a 'date_posted' column
df_tom_glassdoor['Date Posted'] = random_dates

print("Sample of imputed Glassdoor dates:")
print(df_tom_glassdoor[['Date Posted', 'source']].head())

Sample of imputed Glassdoor dates:
  Date Posted     source
0  2025-06-07  Glassdoor
1  2025-05-28  Glassdoor
2  2025-05-14  Glassdoor
3  2025-06-11  Glassdoor
4  2025-05-07  Glassdoor


#### Random Imputation Approach Overview
Distribute the Glassdoor job listings randomly and uniformly across the known date range. This assumes that a job was equally likely to be posted on any given day within that window.

* How to do it: For each row in your Glassdoor data, generate a random date between April 1, 2025, and June 21, 2025.

##### Pros:
* Far more realistic than the midpoint approach. It reflects that postings were spread out over time.
* Avoids creating an artificial spike, leading to more natural-looking time series plots.
* Preserves the total number of jobs within the period.

##### Cons:
* The dates are still synthetic (an educated guess). The true distribution might not have been uniform (e.g., more jobs get posted on Mondays).
* Introduces randomness. You must set a random seed to ensure your results are reproducible every time you run the script.
  
##### Best for: Most standard time series analysis and visualization, where avoiding the artificial spike from Strategy 1 is important. This is often the most balanced and useful approach.

***

### Step 4: Standardise and Combine the Datasets

In [8]:
# Ensure the Adzuna dates column are in datetime format with day-first parsing
df_pengjin_adzuna['Date Posted'] = pd.to_datetime(df_pengjin_adzuna['Date Posted'], dayfirst=True)
df_uni_adzuna['Date Posted'] = pd.to_datetime(df_uni_adzuna['Date Posted'], dayfirst=True)

# Select common columns to ensure a clean merge
common_columns = ['source', 'author','Date Posted', 'Job Title', 'Company Name', 'Location', 'Salary', 'Job Description', 'search_keyword', 
                  'City', 'Country', 'Region', 'salary_currency', 'salary_min', 'salary_max', 'salary_period', 'salary_source', 
                  'salary_standardized', 'median_annual_salary_gbp']


# Concatenate the two dataframes
df_combined = pd.concat([
    df_uni_adzuna[common_columns],
    df_pengjin_adzuna[common_columns],
    df_tom_glassdoor[common_columns]
], ignore_index=True)

print(f"\nTotal rows in combined dataset: {len(df_combined)}")
print("\nValue counts by source:")
print(df_combined['source'].value_counts())
print("\nValue counts by author:")
print(df_combined['author'].value_counts())


Total rows in combined dataset: 10097

Value counts by source:
source
Adzuna       9446
Glassdoor     651
Name: count, dtype: int64

Value counts by author:
author
UoB        8530
Pengjin     916
Tom         651
Name: count, dtype: int64


In [9]:
df_combined.sample(10)

,source,author,Date Posted,Job Title,Company Name,Location,Salary,Job Description,search_keyword,City,Country,Region,salary_currency,salary_min,salary_max,salary_period,salary_source,salary_standardized,median_annual_salary_gbp
4680,Adzuna,UoB,2021-01-18,Senior Embedded Software Engineer,NaN,Warwick,NaN,LINKS Global Tech have a fantastic opportunity...,NaN,Warwick,NaN,West Midlands (England),NaN,NaN,NaN,NaN,NaN,NaN,NaN
447,Adzuna,UoB,2020-09-07,"Senior Software Engineer - Go, Kubernetes, AWS",NaN,"London, SE11","£80,000 - £100,000","Senior Software Engineer - Go, Kubernetes, AWS...",NaN,London,NaN,London,GBP,80000.00,100000.00,annual,NaN,"GBP 80,000 - 100,000 (annual)",90000.00
4565,Adzuna,UoB,2020-11-24,Contract Front End Developer,NaN,Oxfordshire,NaN,Front End Developer required for a short term ...,NaN,Oxfordshire,NaN,South East (England),NaN,NaN,NaN,NaN,NaN,NaN,NaN
9016,Adzuna,Pengjin,2024-08-02,Machine Learning Engineer,Better Placed - A Sunday Times Top 10 Employer!,"Eastern England, UK",NaN,Machine Learning Engineer Remote (UK only) £40...,machine learning,NaN,England,East of England,GBP,55489.14,55489.14,annual,NaN,NaN,55489.14
2751,Adzuna,UoB,2019-01-02,C# Developer .Net SQL,NaN,Guildford,£40k,C# Developer London to £40k C# Developer / Jun...,NaN,Guildford,NaN,South East (England),GBP,40000.00,40000.00,annual,NaN,"GBP 40,000 (annual)",40000.00
911,Adzuna,UoB,2019-04-02,"Android Developer - £65,000 to £75,000 - London",NaN,"London, Greater London","£65,000 to £75,000","Android Developer - £65,000 to £75,000 - Londo...",NaN,London,NaN,London,GBP,65000.00,75000.00,annual,NaN,"GBP 65,000 - 75,000 (annual)",70000.00
2784,Adzuna,UoB,2022-05-06,Business Analyst,NaN,Bradford,"£35,112 - £43,890",Company description: Water Utility Company bas...,NaN,Bradford,NaN,Yorkshire and The Humber,GBP,35112.00,43890.00,annual,NaN,"GBP 35,112 - 43,890 (annual)",39501.00
2982,Adzuna,UoB,2019-10-04,Business Intelligence Analyst/Developer - Bansley,NaN,"England - North East, North Yorkshire","£25,000 - £32,000",My client is based in Barnsley and is looking ...,NaN,England - North East,NaN,North East (England),GBP,25000.00,32000.00,annual,NaN,"GBP 25,000 - 32,000 (annual)",28500.00
6903,Adzuna,UoB,2022-06-22,Data Science Trainer,NaN,London,NaN,Data Science Principal Trainer London - Hybrid...,NaN,London,NaN,London,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7819,Adzuna,UoB,2019-04-04,Software Engineer - Medical Devices,NaN,Cambridgeshire,NaN,An exciting opportunity to join an innovative ...,NaN,Cambridgeshire,NaN,East of England,NaN,NaN,NaN,NaN,NaN,NaN,NaN


***
### Step 5: Handling Missing Salary Data in Uni-provided Adzuna Data

#### Methodology for Handling Missing Salary Data
Given that approximately 60% of the uni-provided Adzuna listings lack salary data, a robust imputation strategy is required to avoid significant data loss and selection bias. We are proceeding with **Grouped Median Imputation**.

#### This methodology involves the following steps:

1. **Stratification**: The dataset is stratified into distinct cohorts based on primary salary predictors: Job Title and Location (Region).
2. **Median Calculation:** The median salary is calculated for each *(Job Title, Region)* cohort using the records where salary data is present. The median is chosen over the mean for its robustness to outliers within each group.
3. **Imputation:** Missing salary values are then populated with the calculated median of their respective cohort.

#### Justification:

This approach is superior to a single global median as it preserves the inherent variance in compensation across different roles and geographic markets. It provides a reasonable and defensible estimate by leveraging the strongest available predictors in the raw data.

To account for cohorts that may lack any salary data, a fallback hierarchy is implemented: first imputing by ***Job Title*** median alone, and finally by the global median if necessary. 

This ensures completeness while maintaining the highest possible level of specificity at each step. This method provides a reliable foundation for subsequent feature engineering and analysis without introducing the confounding complexities of a full predictive model at the preprocessing stage.

In [12]:
# 1. Confirm number of observations with missing salaries
print(f"\nSalaries missing before imputation: {df_combined['median_annual_salary_gbp'].isna().sum()}")


Salaries missing before imputation: 5174


In [13]:
# 2. Create a 'salary_is_imputed' Flag / Variable

# This column will track which values were original vs. imputed for performing a sensitivity analysis later.
df_combined['is_salary_imputed'] = df_combined['median_annual_salary_gbp'].isna()

In [16]:
# 3.1 Create Year column
df_combined['Year'] = df_combined['Date Posted'].dt.year

# --- 3.2 MULTI-TIERED SALARY IMPUTATION ---

print("\n--- Starting Imputation Process ---")

# --- Level 1: Impute using (Year, Region, Job Title) Median ---
print("\nLEVEL 1: Imputing with median of (Year, Region, Job Title) groups...")
group_medians_level1 = df_combined.groupby(['Year', 'Region', 'Job Title'])['median_annual_salary_gbp'].transform('median')
# FIX: Use direct assignment instead of inplace=True
df_combined['median_annual_salary_gbp'] = df_combined['median_annual_salary_gbp'].fillna(group_medians_level1)
print(f"Salaries still missing: {df_combined['median_annual_salary_gbp'].isna().sum()}")


# --- Level 2: Fallback to (Year, Job Title) Median ---
print("\nLEVEL 2 (Fallback): Imputing with median of (Year, Job Title) groups...")
# FIX: Pass column names as a list to groupby
group_medians_level2 = df_combined.groupby(['Year', 'Job Title'])['median_annual_salary_gbp'].transform('median')
df_combined['median_annual_salary_gbp'] = df_combined['median_annual_salary_gbp'].fillna(group_medians_level2)
print(f"Salaries still missing: {df_combined['median_annual_salary_gbp'].isna().sum()}")


# --- Level 3: Fallback to (Year) Median ---
print("\nLEVEL 3 (Fallback): Imputing with median of (Year) groups...")
group_medians_level3 = df_combined.groupby('Year')['median_annual_salary_gbp'].transform('median')
df_combined['median_annual_salary_gbp'] = df_combined['median_annual_salary_gbp'].fillna(group_medians_level3)
print(f"Salaries still missing: {df_combined['median_annual_salary_gbp'].isna().sum()}")


# --- Level 4: Fallback to Global Median ---
print("\nLEVEL 4 (Fallback): Imputing with global median salary...")
global_median = df_combined['median_annual_salary_gbp'].median()
df_combined['median_annual_salary_gbp'] = df_combined['median_annual_salary_gbp'].fillna(global_median)
print(f"Salaries still missing: {df_combined['median_annual_salary_gbp'].isna().sum()}")


# --- 4. VERIFICATION ---
print("-" * 50)
print("\n--- Final DataFrame After Imputation ---")
# Display the result, focusing on the filled values and the flag
print(df_combined)

# Verify the changes for a specific row
print("\nExample: The previously missing salary for a 'Data Scientist' in London:")
print(df_combined.iloc[2])


--- Starting Imputation Process ---

LEVEL 1: Imputing with median of (Year, Region, Job Title) groups...
Salaries still missing: 4392

LEVEL 2 (Fallback): Imputing with median of (Year, Job Title) groups...
Salaries still missing: 3716

LEVEL 3 (Fallback): Imputing with median of (Year) groups...
Salaries still missing: 0

LEVEL 4 (Fallback): Imputing with global median salary...
Salaries still missing: 0
--------------------------------------------------

--- Final DataFrame After Imputation ---
          source author Date Posted  \
0         Adzuna    UoB  2019-07-02   
1         Adzuna    UoB  2022-05-29   
2         Adzuna    UoB  2021-07-02   
3         Adzuna    UoB  2022-04-22   
4         Adzuna    UoB  2019-04-09   
...          ...    ...         ...   
10092  Glassdoor    Tom  2025-05-01   
10093  Glassdoor    Tom  2025-05-04   
10094  Glassdoor    Tom  2025-05-28   
10095  Glassdoor    Tom  2025-06-05   
10096  Glassdoor    Tom  2025-06-06   

                           

In [17]:
# Step 6: Export combined df to csv
df_combined.to_csv("master_cleaned_job_listings_final.csv", index=False)

***
## Data Preparation and Methodology *(for final write-up)*
The objective of the data preparation phase was to consolidate multiple datasets into a single, analysis-ready master file. This required a systematic approach to handle inconsistencies in data structure, harmonize temporal data, and address significant volumes of missing salary information. The following methodology was implemented to ensure the final dataset was both complete and methodologically robust.

#### 1. Data Consolidation and Provenance
The first step involved merging three distinct datasets: df_pengjin_adzuna, df_uni_adzuna, and df_tom_glassdoor. To maximize the statistical power of subsequent steps, the datasets were consolidated before any imputation was performed. This approach ensures that the calculation of descriptive statistics (such as medians for imputation) is based on the largest possible sample size, leading to more stable and reliable estimates.

Crucially, a source column was added to each dataset prior to merging. This preserves data provenance, allowing for clear traceability of each record to its origin. This is a critical step for ensuring transparency and enabling future analysis to compare or control for variations between data sources.

#### 2. Temporal Data Harmonization
A significant challenge was the absence of posting dates in the Glassdoor dataset, contrasted with the historical data from Adzuna dating back to 2019. To prepare the data for time series analysis, the following steps were taken:
    
* Imputation of Missing Dates: For the Glassdoor listings, which were known to be from a two-month window in 2025, dates were imputed by assigning a random uniform date within that known period. This avoids the creation of an artificial data spike on a single day and reflects the reality that postings were distributed over time.

    * Creation of 'Year' Column: A Year column was derived from the posting date of every record. This column is the cornerstone of the time-aware imputation strategy.

#### 3. Missing Salary Data Imputation
A substantial portion of the consolidated dataset (~60%) lacked salary information. Simply deleting these records would have drastically reduced the dataset's size and introduced significant selection bias, as the factors that lead to a salary being listed are non-random. Therefore, a multi-tiered imputation strategy was employed.

#### 3.1. Rationale for Grouped Imputation
Instead of using a single global median, which would obscure significant market variations, a grouped median imputation strategy was chosen. This method operates on the principle that compensation is primarily a function of role, geography, and time.

#### 3.2. A Time-Aware, Multi-Tiered Strategy
To account for wage growth and inflation, the imputation was made time-aware by including the Year as a primary grouping variable. This ensures that a missing 2019 salary is only informed by other 2019 salaries. A multi-level fallback system was designed to maximize data retention while maintaining the highest possible specificity:

1. Level 1 (Most Specific): Missing salaries were first imputed using the median of their specific (Year, Job Title, Region) cohort.
2. Level 2 (Fallback): If a Level 1 cohort had no salary data, imputation was performed using the median of the broader (Year, Job Title) group.
3. Level 3 (Final Fallback): For any remaining missing values, the median of the corresponding Year was used.

The median was chosen over the mean for its robustness to outliers within each group.

#### 3.3. Justification for Geographic Granularity
The Region was chosen for geographic grouping over the City due to data quality considerations. The Region data was clean and consistent, whereas the City data contained variations that would have fragmented the cohorts. Using Region creates larger, more statistically stable groups, providing more reliable medians for imputation while still capturing major economic differences across the UK.

#### 4. Methodological Safeguards
To ensure analytical transparency, an is_salary_imputed boolean flag was created for every record before the imputation process began. This flag allows for a clear distinction between original and imputed data points. Its presence is critical for conducting a sensitivity analysis, whereby the final analytical results can be compared between the full, imputed dataset and the subset of records with original salary data. This step validates the imputation strategy and quantifies its impact on the final conclusions.